_Run the first 2 code cells without modifications_

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import Markdown #, IFrame
# for presentations:
#display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import pandas as pd
#pd.set_option("display.max_colwidth", 200)
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.ion()
plt.style.use('seaborn-v0_8-muted')
from pprint import pprint as ptp

import sys
from pathlib import Path
print('Python ver: {}\nPython env: {}'.format(sys.version, Path(sys.prefix).name))
print('Currrent dir: {}\n'.format(Path.cwd()))


def add_to_sys_path(this_path, up=False):
    """
    Prepend this_path to sys.path.
    If up=True, path refers to parent folder (1 level up).
    """

    if up:
        newp = str(Path(this_path).parent)
    else:
        newp = str(Path(this_path))
    if newp not in sys.path:
        sys.path.insert(1, newp)
        print('Path added to sys.path: {}'.format(newp))


def fdir(obj, start_with_str='_', exclude=True):
    """Filtered dir() for method discovery."""
    return [d for d in dir(obj) if not d.startswith(start_with_str) == exclude]

def despine(which=['top','right']):
    """which ([str])): 'left','top','right','bottom'."""

    ax = plt.gca()
    for side in which:
        ax.spines[side].set_visible(False)
    return

def md_width_comment(w:int=120) -> str:
    """Width guide for composing md documents."""
    return f"<!-- dotted line width = {w}\n{'.'*w}-->"


# autoreload extension
%load_ext autoreload
%autoreload 2

Python ver: 3.11.5 | packaged by conda-forge | (main, Aug 27 2023, 03:34:09) [GCC 12.3.0]
Python env: mce
Currrent dir: /home/cat/projects/MCCE_Benchmarking/notebooks



---
---

In [2]:
from mcce_benchmark import APP_NAME, BENCH
from mcce_benchmark import DEFAULT_DIR, N_BATCH, N_PDBS, ENTRY_POINTS, OUT_FILES, MCCE_OUTPUTS, ANALYZE_DIR

from mcce_benchmark import USER_PRFX, USER_ENV, CONDA_PATH
from mcce_benchmark import io_utils as iou
#from mcce_benchmark import Pathok

import subprocess
from typing import Union
import shutil

In [6]:
from mcce_benchmark import pkanalysis

In [10]:
here = Path.cwd()

# completed runs w/analysis:
benchmarks_dir = iou.Pathok(Path.cwd().parent.parent.joinpath("bench_tests", "all_pdbs"))
print(f"{benchmarks_dir = }")

pdbs = benchmarks_dir.joinpath(BENCH.CLEAN_PDBS)
print(f"{pdbs = }")

benchmarks_dir = PosixPath('/home/cat/projects/bench_tests/all_pdbs')
pdbs = PosixPath('/home/cat/projects/bench_tests/all_pdbs/clean_pdbs')


In [45]:
matched_fp = benchmarks_dir.joinpath(ANALYZE_DIR, OUT_FILES.MATCHED_PKAS.value)
print(f"{matched_fp = }")

matched_df = pkanalysis.load_matched_pkas(matched_fp)
matched_stats = pkanalysis.matched_pkas_stats(matched_df)
type(matched_stats)
matched_stats

matched_fp = PosixPath('/home/cat/projects/bench_tests/all_pdbs/analysis/matched_pkas.csv')


dict

{'fit': (0.9638685701613313, 1.0),
 'N': 1028,
 'mean_delta': 1.0635564202334629,
 'rmsd': 1.7067523709202879,
 'bounds': [3.0, 2.0, 1.0],
 'report': 'Residues stats:\nNumber of pKas matched with those in pKaDB: 1,028\nFit line: y = 0.96.x + 0.09\nMean delta pKa: 1.06\nRMSD, calculated vs experimental: 1.71\nProportion within 3.0 pH units: 94.55%\nProportion within 2.0 pH units: 88.13%\nProportion within 1.0 pH units: 64.88%\n'}

In [46]:
print(matched_stats["report"])

Residues stats:
Number of pKas matched with those in pKaDB: 1,028
Fit line: y = 0.96.x + 0.09
Mean delta pKa: 1.06
RMSD, calculated vs experimental: 1.71
Proportion within 3.0 pH units: 94.55%
Proportion within 2.0 pH units: 88.13%
Proportion within 1.0 pH units: 64.88%



In [22]:
matched_stats_fp = matched_fp.parent.joinpath(OUT_FILES.MATCHED_PKAS_STATS.value)
dict_to_json(matched_stats, matched_stats_fp)
print(f"{matched_stats_fp = }")

matched_stats_fp = PosixPath('/home/cat/projects/bench_tests/all_pdbs/analysis/matched_pkas_stats.json')


In [24]:
matched_stats_fromjson = json_to_dict(matched_stats_fp)
type(matched_stats_fromjson)

dict

In [35]:
rpt = report_matched_stats(matched_stats)

In [36]:
print(rpt)

Residues statistics:
fit: (0.96, 1.0)
N: 1028
mean_delta: 1.06
rmsd: 1.71
bounds: [3.0, 2.0, 1.0]
txt: Residues stats:
Number of pKas matched with those in pKaDB: 1,028
Fit line: y = 0.96.x + 0.09
Mean delta pKa: 1.06
RMSD, calculated vs experimental: 1.71
Proportion within 3.0 pH units: 94.55%
Proportion within 2.0 pH units: 88.13%
Proportion within 1.0 pH units: 64.88%




---
---
# Refset prep:

---
---

---
# Test cli

In [176]:
from argparse import ArgumentParser, ArgumentError, RawDescriptionHelpFormatter, Namespace as argNamespace
from mcce_benchmark import cli

In [177]:
clilogr = cli.logger
clilogr.hasHandlers()

In [179]:
cli_parser = cli.bench_parser()

cli_parser.print_help()

usage: 
bench_expl_pkas <+ 1 sub-command: setup_job or launch_job> <related args>

Examples:
1. setup_job: Data & script setup:
   - Minimal input: value for -benchmarks_dir option:
     >bench_expl_pkas setup_job -benchmarks_dir <folder name>

   - Using non-default option(s):
     >bench_expl_pkas setup_job -benchmarks_dir <folder name> -d 8

2. launch_job: Launch runs:
   - Minimal input: value for -benchmarks_dir option:
     >bench_expl_pkas launch_job -benchmarks_dir <folder name>

   - Using non-default option(s):
     >bench_expl_pkas launch_job -benchmarks_dir <folder name> -n_active <jobs to maintain>
     >bench_expl_pkas launch_job -benchmarks_dir <folder name> -job_name <my_job_name> -sentinel_file step2_out.pdb

Description:
Launch a MCCE benchmarking job using curated structures from the pKa Database v1.

The main command is bench_expl_pkas along with one of 2 sub-commands:
- Sub-command 1: setup_job: setup the dataset and run script to run mcce steps 1 through 4;
- Sub-

---
## Test help msg


## cli: None input

In [128]:
cmd = None
cli.bench_cli(cmd)


Description:
Launch a MCCE benchmarking job using curated structures from the pKa Database v1.

The main command is bench_expl_pkas along with one of 2 sub-commands:
- Sub-command 1: setup_job: setup the dataset and run script to run mcce steps 1 through 4;
- Sub-command 2: launch_job: launch a batch of jobs;


bench_expl_pkas <+ 1 sub-command: setup_job or launch_job> <related args>

Examples:
1. setup_job: Data & script setup:
   - Minimal input: value for -benchmarks_dir option:
     >bench_expl_pkas setup_job -benchmarks_dir <folder name>

   - Using non-default option(s):
     >bench_expl_pkas setup_job -benchmarks_dir <folder name> -d 8

2. launch_job: Launch runs:
   - Minimal input: value for -benchmarks_dir option:
     >bench_expl_pkas launch_job -benchmarks_dir <folder name>

   - Using non-default option(s):
     >bench_expl_pkas launch_job -benchmarks_dir <folder name> -n_active <jobs to maintain>
     >bench_expl_pkas launch_job -benchmarks_dir <folder name> -job_name <m

In [129]:
cmd = []
cli.bench_cli(cmd)


Description:
Launch a MCCE benchmarking job using curated structures from the pKa Database v1.

The main command is bench_expl_pkas along with one of 2 sub-commands:
- Sub-command 1: setup_job: setup the dataset and run script to run mcce steps 1 through 4;
- Sub-command 2: launch_job: launch a batch of jobs;


bench_expl_pkas <+ 1 sub-command: setup_job or launch_job> <related args>

Examples:
1. setup_job: Data & script setup:
   - Minimal input: value for -benchmarks_dir option:
     >bench_expl_pkas setup_job -benchmarks_dir <folder name>

   - Using non-default option(s):
     >bench_expl_pkas setup_job -benchmarks_dir <folder name> -d 8

2. launch_job: Launch runs:
   - Minimal input: value for -benchmarks_dir option:
     >bench_expl_pkas launch_job -benchmarks_dir <folder name>

   - Using non-default option(s):
     >bench_expl_pkas launch_job -benchmarks_dir <folder name> -n_active <jobs to maintain>
     >bench_expl_pkas launch_job -benchmarks_dir <folder name> -job_name <m

## cli: 1 input

In [130]:
cmd = f"{cli.SUB_CMD1}".split()
cmd
cli.bench_cli(cmd)

['setup_job']


Sub-command for setting up <benchmarks_dir>/clean_pdbs folder & job_name_run.sh script, e.g.:
>bench_expl_pkas setup_job -benchmarks_dir <folder name>



In [131]:
cmd = f"{cli.SUB_CMD2}".split() #benchmarks_dir {benchmarks_dir} -job_name {job} -d 8".split()

cli.bench_cli(cmd)

Sub-command for launching a batch of jobs, e.g.:
>bench_expl_pkas launch_job -benchmarks_dir <folder name> -n_active 15
Note: if provided, the value for the -job_name option must match the one used in `setup_job`.



## cli: 2 with help

In [132]:
cmd = f"{cli.SUB_CMD1} -h".split() #benchmarks_dir {benchmarks_dir} -job_name {job} -d 8".split()
cmd
#args = cli_parser.parse_args(cmd)

cli.bench_cli(cmd)

['setup_job', '-h']


Description:
Launch a MCCE benchmarking job using curated structures from the pKa Database v1.

The main command is bench_expl_pkas along with one of 2 sub-commands:
- Sub-command 1: setup_job: setup the dataset and run script to run mcce steps 1 through 4;
- Sub-command 2: launch_job: launch a batch of jobs;


bench_expl_pkas <+ 1 sub-command: setup_job or launch_job> <related args>

Examples:
1. setup_job: Data & script setup:
   - Minimal input: value for -benchmarks_dir option:
     >bench_expl_pkas setup_job -benchmarks_dir <folder name>

   - Using non-default option(s):
     >bench_expl_pkas setup_job -benchmarks_dir <folder name> -d 8

2. launch_job: Launch runs:
   - Minimal input: value for -benchmarks_dir option:
     >bench_expl_pkas launch_job -benchmarks_dir <folder name>

   - Using non-default option(s):
     >bench_expl_pkas launch_job -benchmarks_dir <folder name> -n_active <jobs to maintain>
     >bench_expl_pkas launch_job -benchmarks_dir <folder name> -job_name <m

## cli: setup with n_pdbs

In [133]:
from mcce_benchmark import custom_sh

In [162]:
ben_dir = "../foo_dir"
cmd = f"{cli.SUB_CMD1} -benchmarks_dir {ben_dir} -n_pdbs 2".split() # -job_name {job} -d 8".split()
cmd
args = cli_parser.parse_args(cmd)

all_default = custom_sh.all_opts_are_defaults(args)
all_default

#cli.bench_cli(cmd)

[INFO]: mcce_benchmark.cli, <module>:
	
START
----------------------------------------------------------------------
2024-02-28 13:41:41 - USER = 'cat' - User envir: mce
APP VER: (0, 1, 0, 'dev5', 'g2095b87.d20240226')
APP DEFAULTS:
Globals: MCCE_EPS = 4; N_ACTIVE = 10
Default resource names:
DEFAULT_DIR = 'mcce_benchmarks' : default benchmarking folder name
BENCH.CLEAN_PDBS = 'clean_pdbs' : fixed
BENCH.Q_BOOK = 'book.txt' : jobs bookkeeping file
BENCH.DEFAULT_JOB = 'default_run' (-> default_run.sh script in clean_pdbs/)
BENCH.BENCH_PARSE_E4 = PosixPath('/home/cat/projects/MCCE_Benchmarking/mcce_benchmark/data/refsets/parse.e4') : Current reference set
N_PDBS = 120 : number of pdbs in the dataset
Default analysis output file names (fixed):
OUT_FILES.MATCHED_PKAS_FILE.name = 'MATCHED_PKAS_FILE'
OUT_FILES.ALL_PKAS_FILE.name = 'ALL_PKAS_FILE'
OUT_FILES.CONF_COUNTS.name = 'CONF_COUNTS'
OUT_FILES.RES_COUNTS.name = 'RES_COUNTS'
OUT_FILES.RUN_TIMES.name = 'RUN_TIMES'
OUT_FILES.CONFS_PER_RES.n

['setup_job', '-benchmarks_dir', '../foo_dir', '-n_pdbs', '2']

True

In [138]:
cmd = f"{cli.SUB_CMD2} -benchmarks_dir {ben_dir}".split()
cmd

cli.bench_cli(cmd)

['launch_job', '-benchmarks_dir', '../foo_dir']

[INFO]: mcce_benchmark.cli, bench_launch_batch:
	bench_expl_pkas args:
{'subparser_name': 'launch_job',
 'benchmarks_dir': PosixPath('/home/cat/projects/MCCE_Benchmarking/notebooks/foo_dir'),
 'job_name': 'default_run',
 'n_active': 10,
 'sentinel_file': 'pK.out',
 'func': <function mcce_benchmark.cli.bench_launch_batch(args: argparse.Namespace) -> None>}

[INFO]: mcce_benchmark.cli, bench_launch_batch:
	Script contents prior to launch:
#!/bin/bash

step1.py --dry prot.pdb
step2.py -d 4
step3.py -d 4
step4.py --xts

sleep 10

[INFO]: mcce_benchmark.cli, bench_launch_batch:
	Submiting batch of jobs.
[INFO]: mcce_benchmark.scheduling, create_cron_sh:
	Created script for crontab 'crontab_default_run_sh' in /home/cat/projects/MCCE_Benchmarking/notebooks/foo_dir

[INFO]: mcce_benchmark.scheduling, schedule_job:
	Created the bash script for crontab.
[INFO]: mcce_benchmark.scheduling, create_crontab:
	Crontab text:
#Scheduled from bench_launchjob
* * * * * /home/cat/projects/MCCE_Benchmarking

---
## Scheduling

In [141]:
from mcce_benchmark import scheduling

[INFO]: mcce_benchmark.cli, <module>:
	
START
----------------------------------------------------------------------
2024-02-28 13:41:41 - USER = 'cat' - User envir: mce
APP VER: (0, 1, 0, 'dev5', 'g2095b87.d20240226')
APP DEFAULTS:
Globals: MCCE_EPS = 4; N_ACTIVE = 10
Default resource names:
DEFAULT_DIR = 'mcce_benchmarks' : default benchmarking folder name
BENCH.CLEAN_PDBS = 'clean_pdbs' : fixed
BENCH.Q_BOOK = 'book.txt' : jobs bookkeeping file
BENCH.DEFAULT_JOB = 'default_run' (-> default_run.sh script in clean_pdbs/)
BENCH.BENCH_PARSE_E4 = PosixPath('/home/cat/projects/MCCE_Benchmarking/mcce_benchmark/data/refsets/parse.e4') : Current reference set
N_PDBS = 120 : number of pdbs in the dataset
Default analysis output file names (fixed):
OUT_FILES.MATCHED_PKAS_FILE.name = 'MATCHED_PKAS_FILE'
OUT_FILES.ALL_PKAS_FILE.name = 'ALL_PKAS_FILE'
OUT_FILES.CONF_COUNTS.name = 'CONF_COUNTS'
OUT_FILES.RES_COUNTS.name = 'RES_COUNTS'
OUT_FILES.RUN_TIMES.name = 'RUN_TIMES'
OUT_FILES.CONFS_PER_RES.n

In [100]:
fdir(scheduling)

['CONDA_PATH',
 'CRON_COMMENT',
 'CRON_SH_PREFIX',
 'CronTab',
 'ENTRY_POINTS',
 'Path',
 'Pathok',
 'USER',
 'USER_ENV',
 'USER_PRFX',
 'Union',
 'argNamespace',
 'create_cron_sh',
 'create_crontab',
 'create_crontab_old',
 'logger',
 'logging',
 'make_executable',
 'schedule_job',
 'shutil',
 'subprocess',
 'subprocess_run',
 'sys']

In [103]:
USER_PRFX, USER_ENV, CONDA_PATH
ENTRY_POINTS["launch"]
benchmarks_dir

('/home/cat/miniconda3/envs/mce', 'mce', '/home/cat/miniconda3/condabin/conda')

'mccebench_launchjob'

PosixPath('/home/cat/projects/mcce_benchmarks_test')

In [ ]:
cron_txt = scheduling.create_single_crontab( benchmarks_dir, job, debug=True)
print(cron_txt)

---
# Test cli

In [ ]:
logger = logging.getLogger()

In [242]:
job = "norun_foo"
custom_sh.write_run_script_from_template(benchmarks_dir, job,
                                                    script_template = ScriptChoices.NORUN,
                                                    job_args = None)
sh_path = benchmarks_dir.joinpath(BENCH.CLEAN_PDBS, f"{job}.sh")
sh_path.exists()
sh_path
!cat {sh_path}

True

PosixPath('/home/cat/projects/mcce_benchmarks_test/clean_pdbs/norun_foo.sh')


#!/bin/bash

step1.py prot.pdb --norun


In [231]:
!cat {sh_path}


 #!/bin/bash

 step1.py prot.pdb --dry -d 8.0
 step2.py -d 8.0
 step3.py -d 8.0
 step4.py --xts 

 sleep 10


In [126]:
use_default_script = check_steps_opt_defaults(args)
use_default_script

True

In [205]:
out = populate_custom_template(args)
print(out)



 #!/bin/bash

 step1.py prot.pdb --dry -d 8.0
 step2.py -d 8.0
 step3.py -d 8.0
 step4.py --xts 

 sleep 10

